In [6]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/

In [7]:
import logging
import delta_sharing
import pandas as pd

import general_functions.databricks_client as db_client
from general_functions.return_account_ids import return_account_ids
from general_functions.constants import return_api_url
from general_functions.call_api_with_account_id import call_api_with_accountId, send_to_innkeepr_api_paginated

In [ ]:
customer = "ESN"
url = return_api_url()
print(f"url = {url}")
account_id = return_account_ids()
account_id = [acc["id"] for acc in account_id if acc["name"] == customer]
account_id = account_id[0]

In [9]:
profile_path = db_client.return_databricks_client()
table_path = f"{profile_path}#delta_share_events.{account_id}.features_view_30_outlook" #features_view_30_outlook"
df = delta_sharing.load_as_pandas(table_path, limit=20000)

In [10]:
df.to_parquet("/Users/karolinegriesbach/Documents/Innkeepr/Git/Analysis/T3_Smartbidding_Updates/data/targeting_history/LILLYDOO/with_db_view_67d1b8102eb78f8bcc5e2c57/feat_views_conversions.parquet")

In [5]:
df["created"].min(), df["created"].max()

In [7]:
# df = df[["created","session","anonymousId","session_duration_in_s","landingpage","landingpage_cat","pages_visited_per_user_cumulated","pages_visited_per_user","pages_visited_per_session_cumulated","pages_visited_per_session","conv_name","treatment"]].sort_values(by=["anonymousId","created"])
# df = df.reset_index(drop=True)
df

In [8]:
df[df["url"].str.contains("trial")]["url"]

In [7]:
df.to_parquet(f"DataChecks/views/{customer}.parquet")
df.iloc[0:100].to_csv(f"DataChecks/views/{customer}.csv")

In [8]:
df.groupby("date")["anonymousId"].nunique().sort_index()

In [9]:
df.drop_duplicates(subset=["session","conv_name"]).groupby("date")["conv_name"].value_counts()

In [10]:
df[df["url"].str.contains("innkeepr_id")][["created","url"]].sort_values(by="created")

In [11]:
df[df["url"].str.contains("ad_id")][["created","url"]].sort_values(by="created")

In [12]:
df[(df["url"].str.contains("innkeepr_id"))|(df["url"].str.contains("ad_id"))][["created","url"]].sort_values(by="created")

In [13]:
import numpy as np
#temp = df[(df["url"].str.contains("innkeepr_id"))|(df["url"].str.contains("ad_id"))]
df["google_id"] = None
df["google_id"] = np.where(df["url"].str.contains("innkeepr_id"),"innkeepr_id",df["google_id"])
df["google_id"] = np.where(df["url"].str.contains("innkepr_id"),"innkepr_id",df["google_id"])
df["google_id"] = np.where(df["url"].str.contains("ad_id"),"ad_id",df["google_id"])
unique_ids = df.groupby("treatment")["google_id"].unique()
unique_ids = unique_ids.reset_index()
nuniqe_ids = df.groupby("treatment")["google_id"].nunique()
nuniqe_ids = nuniqe_ids.reset_index()
check_ids = pd.merge(unique_ids, nuniqe_ids, on="treatment", suffixes=("","_count"))
check_ids.sort_values(by="google_id_count", ascending=False)

In [14]:
treatments = send_to_innkeepr_api_paginated(f"{url}/treatments/query",account_id,{"id":check_ids[check_ids["treatment"]!="NaN"]["treatment"].dropna().tolist()},logging)
treatments = pd.json_normalize(treatments)

In [15]:
check_ids_merged = pd.merge(
    check_ids,
    treatments[["id","name","relates_to.campaign.name","relates_to.adset.name","source","relates_to.assetGroup.name"]],
    left_on="treatment",
    right_on="id",
    how="left"
)
check_ids_merged[].sort_values(by="google_id_count", ascending=False)